<a href="https://colab.research.google.com/github/xinxinxu2025/code_memo/blob/main/Python%E5%AE%9F%E8%B7%B5%E3%82%AC%E3%82%A4%E3%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

小売店のデータを模擬した練習用DataFrameを作成します。

In [33]:
# バージョン管理（最初に必ず確認）
import pandas as pd
print(f"Pandas version: {pd.__version__}")

# 再現性の確保（乱数のシード設定）
import numpy as np
np.random.seed(42)  # 結果を再現可能にする

Pandas version: 2.2.2


In [18]:
import os

# 現在の作業ディレクトリ
print("Current working directory:", os.getcwd())

# ファイル一覧の表示
print("Files in current directory:", os.listdir('.'))

Current working directory: /content
Files in current directory: ['.config', 'sample_data.csv', 'sample_data']


In [32]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# 乱数のシードを設定（再現性のため）
np.random.seed(42)
random.seed(42)

# データサイズの設定
n_rows = 1000

# 基本データの生成
dates = [datetime(2023, 1, 1) + timedelta(days=i) for i in range(n_rows)]
store_ids = [f'Store_{i:03d}' for i in range(1, 11)]
product_categories = ['Electronics', 'Clothing', 'Food', 'Books', 'Home']
regions = ['Tokyo', 'Osaka', 'Nagoya', 'Fukuoka', 'Sapporo']
payment_methods = ['Credit Card', 'Cash', 'E-money', 'QR Code']

# 売上データの生成
data = {
    'order_id': [f'ORD_{i:05d}' for i in range(1, n_rows + 1)],
    'order_date': dates,
    'store_id': np.random.choice(store_ids, n_rows),
    'product_category': np.random.choice(product_categories, n_rows, p=[0.2, 0.3, 0.25, 0.1, 0.15]),
    'product_name': [],
    'quantity': np.random.randint(1, 11, n_rows),
    'unit_price': [],
    'total_sales': [],
    'customer_id': [f'CUST_{np.random.randint(1000, 9999)}' for _ in range(n_rows)],
    'customer_age': np.random.randint(18, 80, n_rows),
    'customer_gender': np.random.choice(['Male', 'Female', 'Other'], n_rows, p=[0.45, 0.45, 0.1]),
    'region': np.random.choice(regions, n_rows),
    'payment_method': np.random.choice(payment_methods, n_rows, p=[0.4, 0.3, 0.2, 0.1]),
    'discount_rate': np.random.choice([0, 0.1, 0.2, 0.3], n_rows, p=[0.6, 0.2, 0.15, 0.05]),
    'return_flag': np.random.choice([0, 1], n_rows, p=[0.9, 0.1]),
    'rating': np.random.choice([1, 2, 3, 4, 5, np.nan], n_rows, p=[0.05, 0.1, 0.15, 0.3, 0.3, 0.1])
}

# 商品名と単価の設定（カテゴリごとに異なる）
category_prices = {
    'Electronics': (5000, 200000),
    'Clothing': (1000, 15000),
    'Food': (100, 5000),
    'Books': (500, 3000),
    'Home': (2000, 10000)
}

category_products = {
    'Electronics': ['Smartphone', 'Laptop', 'Headphones', 'Tablet', 'Camera'],
    'Clothing': ['T-Shirt', 'Jeans', 'Jacket', 'Dress', 'Shoes'],
    'Food': ['Lunch Box', 'Snacks', 'Drinks', 'Bread', 'Dessert'],
    'Books': ['Novel', 'Textbook', 'Manga', 'Cookbook', 'Magazine'],
    'Home': ['Table', 'Chair', 'Lamp', 'Cookware', 'Decor']
}

# 商品名と単価を生成
for i in range(n_rows):
    category = data['product_category'][i]
    product_name = random.choice(category_products[category])
    min_price, max_price = category_prices[category]
    unit_price = random.randint(min_price, max_price)

    data['product_name'].append(product_name)
    data['unit_price'].append(unit_price)

    # 総売上額を計算（数量×単価×（1-割引率））
    quantity = data['quantity'][i]
    discount = data['discount_rate'][i]
    total_sales = quantity * unit_price * (1 - discount)
    data['total_sales'].append(total_sales)

# DataFrameの作成
df = pd.DataFrame(data)

# 意図的に欠損値を作成（実務を想定）
missing_indices = random.sample(range(n_rows), 50)
for idx in missing_indices:
    col = random.choice(['customer_age', 'rating', 'discount_rate'])
    df.at[idx, col] = np.nan

# 外れ値の追加（実務を想定）
outlier_indices = random.sample(range(n_rows), 10)
for idx in outlier_indices:
    df.at[idx, 'quantity'] = random.randint(50, 100)  # 異常な数量
    df.at[idx, 'unit_price'] = random.randint(500000, 1000000)  # 異常な単価

# データ型の設定（CSV保存前の最適化）
df['order_date'] = pd.to_datetime(df['order_date'])

# CSVファイルとして保存
csv_file_path = 'sales_data.csv'

# 日本語環境でも読みやすい形式で保存
df.to_csv(
    csv_file_path,
    index=False,               # インデックスは保存しない
    encoding='utf-8-sig',      # Excelで開く場合の文字化け防止
    date_format='%Y-%m-%d'     # 日付のフォーマット指定
)

print("✅ sales_data.csv の保存が完了しました！")
print(f"保存先: {csv_file_path}")
print(f"ファイルサイズ: {len(df)}")
print(f"列数: {len(df.columns)}")

# 保存したファイルの情報を表示
print("\n📋 保存されたデータの情報:")
print(f"行数: {len(df)}")
print(f"列数: {len(df.columns)}")
print("\n列一覧:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

print("\nデータ型の内訳:")
print(df.dtypes.value_counts())

✅ sales_data.csv の保存が完了しました！
保存先: sales_data.csv
ファイルサイズ: 1000
列数: 16

📋 保存されたデータの情報:
行数: 1000
列数: 16

列一覧:
 1. order_id
 2. order_date
 3. store_id
 4. product_category
 5. product_name
 6. quantity
 7. unit_price
 8. total_sales
 9. customer_id
10. customer_age
11. customer_gender
12. region
13. payment_method
14. discount_rate
15. return_flag
16. rating

データ型の内訳:
object            8
float64           4
int64             3
datetime64[ns]    1
Name: count, dtype: int64


1. データ読み込みと確認  




In [27]:
import pandas as pd
import numpy as np

try:
    df = pd.read_csv(
        'sales_data.csv',
        encoding='utf-8-sig',  # BOM付きUTF-8に対応
        low_memory=False,      # メモリ効率より正確性を優先
        na_values=['', 'NULL', 'NA', 'N/A', 'null', 'nan', 'NaN', '#N/A'],
        keep_default_na=True
    )
    print("✅ データ読み込み成功")
except FileNotFoundError:
    print("❌ ファイルが見つかりません")
except UnicodeDecodeError:
    # 文字化けした場合のフォールバック
    df = pd.read_csv('sales_data.csv', encoding='shift_jis')
except Exception as e:
    print(f"❌ 予期せぬエラー: {e}")

✅ データ読み込み成功


In [28]:
# よく使うオプション集
# df = pd.read_csv(
#     'data.csv',
#     encoding='utf-8',
#     sep=',',                  # 区切り文字明示指定
#     header=0,                 # ヘッダー行指定
#     skiprows=3,               # 不要なヘッダー行をスキップ
#     skipfooter=2,             # フッターの不要行をスキップ
#     thousands=',',            # カンマ区切り数値対応
#     decimal='.',              # 小数点の形式指定
#     parse_dates=['order_date'], # 日付列の自動変換
#     dayfirst=False,           # 日付形式の指定
#     dtype={                   # データ型の明示的指定
#         'store_id': 'category',
#         'product_id': 'string'
#     },
#     usecols=lambda x: x not in ['temp_column', 'unused_col'],  # 不要列の除外
#     nrows=1000 if DEBUG else None  # デバッグ時はデータ制限
# )

データ確認の必須チェック項目
データを読み込んだら、必ず以下の確認を行いましょう。

In [30]:
# ① 最初のデータを確認

# 最初の5行を表示（データの雰囲気をつかむ）
print("データの先頭5行:")
print(df.head())

# 最後の3行を確認
print("\nデータの最後の3行:")
print(df.tail(3))



# ② データの基本情報を確認

# データの形状（行数と列数）を確認
print(f"データの形状: {df.shape}")
print(f"行数: {df.shape[0]}, 列数: {df.shape[1]}")

# 各列のデータ型と非NULL値の数を確認
print("\nデータの基本情報:")
df.info()



# ③ 数値データの統計量を確認

# 数値列の基本統計量（平均、標準偏差、最小値、最大値など）
print("数値データの統計量:")
print(df.describe())

# オブジェクト型（文字列）の要約統計量
print("\n文字列データの要約:")
print(df.describe(include='object'))



# ④ 列名とデータ型の確認

# 列名の一覧を確認
print("列名一覧:")
print(df.columns.tolist())

# 各列のデータ型を確認
print("\nデータ型一覧:")
print(df.dtypes)



# ⑤ 欠損値の確認

# 各列の欠損値の数を確認
print("各列の欠損値数:")
print(df.isnull().sum())

# 欠損値の割合を確認（％表示）
print("\n欠損値の割合:")
print((df.isnull().sum() / len(df) * 100).round(2))

データの先頭5行:
    order_id  order_date   store_id product_category product_name  quantity  \
0  ORD_00001  2023-01-01  Store_007         Clothing      T-Shirt         8   
1  ORD_00002  2023-01-02  Store_004             Food       Drinks         8   
2  ORD_00003  2023-01-03  Store_008             Food       Snacks         9   
3  ORD_00004  2023-01-04  Store_005      Electronics   Smartphone         7   
4  ORD_00005  2023-01-05  Store_007             Home        Decor         1   

   unit_price  total_sales customer_id  customer_age customer_gender   region  \
0        1409      11272.0   CUST_9851          29.0            Male  Sapporo   
1        2106      16848.0   CUST_1159          66.0           Other    Osaka   
2        1243      11187.0   CUST_7555          34.0           Other    Osaka   
3      182392    1149069.6   CUST_9244          59.0          Female  Sapporo   
4        2712       2169.6   CUST_7846          47.0          Female    Tokyo   

  payment_method  discount_r

In [29]:
# 最初の5ステップ確認法

def initial_data_assessment(df, name="データ"):
    """プロが最初に行う5ステップ確認"""

    print(f"🔍 {name} 初期評価")
    print("=" * 50)

    # ステップ1: 基本形状
    print(f"1. データ形状: {df.shape} (行×列)")
    print(f"   メモリ使用量: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

    # ステップ2: データ型確認
    print(f"2. データ型分布:")
    dtype_counts = df.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"   {dtype}: {count}列")

    # ステップ3: 欠損値状況
    missing_total = df.isnull().sum().sum()
    missing_percent = (missing_total / (df.shape[0] * df.shape[1])) * 100
    print(f"3. 欠損値: {missing_total}箇所 ({missing_percent:.2f}%)")

    # ステップ4: 重複チェック
    duplicates = df.duplicated().sum()
    print(f"4. 完全重複行: {duplicates}行")

    # ステップ5: 基本統計量
    print("5. 数値列の基本統計:")
    print(df.describe().round(2))

# 実行例
initial_data_assessment(df, "売上データ")

🔍 売上データ 初期評価
1. データ形状: (1000, 16) (行×列)
   メモリ使用量: 0.5 MB
2. データ型分布:
   object: 9列
   float64: 4列
   int64: 3列
3. 欠損値: 137箇所 (0.86%)
4. 完全重複行: 0行
5. 数値列の基本統計:
       quantity  unit_price  total_sales  customer_age  discount_rate  \
count   1000.00     1000.00      1000.00        985.00         982.00   
mean       6.18    30968.47    123840.35         49.10           0.07   
std        7.81    86334.94    264773.43         17.96           0.09   
min        1.00      113.00       113.00         18.00           0.00   
25%        3.00     2436.50      8821.58         34.00           0.00   
50%        5.00     4918.00     22969.45         49.00           0.00   
75%        8.00    12837.50     71988.75         65.00           0.10   
max       95.00   963599.00   1705167.00         79.00           0.30   

       return_flag  rating  
count      1000.00  896.00  
mean          0.10    3.78  
std           0.31    1.19  
min           0.00    1.00  
25%           0.00    3.00  
50%      

In [31]:
# 詳細なデータ品質チェック

def detailed_quality_check(df):
    """データ品質の詳細チェック"""

    results = {}

    # 各列の詳細分析
    for column in df.columns:
        col_info = {
            'データ型': str(df[column].dtype),
            'ユニーク値数': df[column].nunique(),
            '欠損値数': df[column].isnull().sum(),
            '欠損率(%)': (df[column].isnull().sum() / len(df) * 100).round(2),
            'サンプル値': df[column].iloc[0] if not df[column].empty else None
        }

        # 数値列の追加チェック
        if pd.api.types.is_numeric_dtype(df[column]):
            col_info.update({
                '最小値': df[column].min(),
                '最大値': df[column].max(),
                '平均値': df[column].mean(),
                'ゼロ値数': (df[column] == 0).sum()
            })

        results[column] = col_info

    return pd.DataFrame(results).T

# 品質チェックの実行
quality_report = detailed_quality_check(df)
print(quality_report)

                     データ型 ユニーク値数 欠損値数 欠損率(%)       サンプル値    最小値        最大値  \
order_id           object   1000    0    0.0   ORD_00001    NaN        NaN   
order_date         object   1000    0    0.0  2023-01-01    NaN        NaN   
store_id           object     10    0    0.0   Store_007    NaN        NaN   
product_category   object      5    0    0.0    Clothing    NaN        NaN   
product_name       object     25    0    0.0     T-Shirt    NaN        NaN   
quantity            int64     20    0    0.0           8      1         95   
unit_price          int64    984    0    0.0        1409    113     963599   
total_sales       float64    996    0    0.0     11272.0  113.0  1705167.0   
customer_id        object    944    0    0.0   CUST_9851    NaN        NaN   
customer_age      float64     62   15    1.5        29.0   18.0       79.0   
customer_gender    object      3    0    0.0        Male    NaN        NaN   
region             object      5    0    0.0     Sapporo    NaN 

よくあるデータ問題と対処法  
・文字化け問題  
・メモリ問題  

In [ ]:
# 文字化け対策の関数
def safe_read_csv(file_path):
    encodings = ['utf-8', 'utf-8-sig', 'shift_jis', 'cp932', 'euc-jp']

    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding, nrows=5)
            print(f"✅ 成功: {encoding}")
            return pd.read_csv(file_path, encoding=encoding)
        except UnicodeDecodeError:
            continue

    print("❌ 適切なエンコーディングが見つかりませんでした")
    return None

# 使用例
df = safe_read_csv('sales_data.csv')

In [ ]:
# 大規模データ用の読み込み
def memory_efficient_read(file_path):
    # まずは軽く読み込んでデータ型を確認
    sample_df = pd.read_csv(file_path, nrows=1000)

    # 最適なデータ型を決定
    dtype_dict = {}
    for col in sample_df.columns:
        if sample_df[col].dtype == 'object':
            # カテゴリ型に変換可能かチェック
            unique_ratio = sample_df[col].nunique() / len(sample_df)
            dtype_dict[col] = 'category' if unique_ratio < 0.5 else 'string'
        else:
            dtype_dict[col] = sample_df[col].dtype

    # 最適なデータ型で読み込み
    return pd.read_csv(file_path, dtype=dtype_dict, low_memory=False)

チェックリスト

読み込み前の確認事項
- [ ] ファイルサイズの確認
- [ ] エンコーディングの予想
- [ ] 必要な列の特定
- [ ] データ型の計画

読み込み時の確認事項
- [ ] エラーハンドリングの実装
- [ ] 適切なデータ型の指定
- [ ] 不要な列の除外
- [ ] 日付列の適切な処理

読み込み後の確認事項
- [ ] データ形状の確認
- [ ] 欠損値の状況把握
- [ ] データ型の再確認
- [ ] 基本統計量の確認
- [ ] サンプルデータの目視確認


In [ ]:
# データの軽量化
def reduce_memory_usage(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        elif df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
    return df

# 進行状況の表示
from tqdm import tqdm
tqdm.pandas()  # apply処理の進捗表示用

3. データクリーニング

データクリーニングは難しく考えず、**基本的なことから始めよう**：

1. **コピーを作る** - 元データを守る
2. **空欄を埋める** - 欠損値処理
3. **変な数字を直す** - 外れ値処理
4. **重複を消す** - 重複削除
5. **型を整える** - データ型整理

In [35]:
# 欠損値の確認
# df.isnull().sum()

# 欠損値を平均値で埋める
# df['column_name'].fillna(df['column_name'].mean(), inplace=True)

# 重複行の削除
# df.drop_duplicates(inplace=True)

# 列名の変更
# df.rename(columns={'old_name': 'new_name'}, inplace=True)

欠損値の処理 - 空欄を埋めよう

In [41]:
# データをコピー（元データを残す）
df_clean = df.copy()

print("欠損値の数を確認:")
print(df_clean.isnull().sum())

# 数値の列の空欄を平均値で埋める
numeric_cols = ['customer_age', 'rating', 'discount_rate']
for col in numeric_cols:
    if col in df_clean.columns:
        avg_value = df_clean[col].mean()
        df_clean[col] = df_clean[col].fillna(avg_value)
        print(f"{col}の空欄を平均値({avg_value:.1f})で埋めました")

# 文字の列の空欄を'不明'で埋める
text_cols = ['customer_gender', 'payment_method']
for col in text_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna('不明')
        print(f"{col}の空欄を'不明'で埋めました")

欠損値の数を確認:
order_id              0
order_date            0
store_id              0
product_category      0
product_name          0
quantity              0
unit_price            0
total_sales           0
customer_id           0
customer_age         15
customer_gender       0
region                0
payment_method        0
discount_rate        18
return_flag           0
rating              104
dtype: int64
customer_ageの空欄を平均値(49.1)で埋めました
ratingの空欄を平均値(3.8)で埋めました
discount_rateの空欄を平均値(0.1)で埋めました
customer_genderの空欄を'不明'で埋めました
payment_methodの空欄を'不明'で埋めました


外れ値の処理 - 変な数字を直そう

In [42]:
print("\n 外れ値チェック:")

# 数量の異常値を見つける
q_low = df_clean['quantity'].quantile(0.01)  # 下1%
q_high = df_clean['quantity'].quantile(0.99) # 上1%

print(f"数量の正常範囲: {q_low} ～ {q_high}")

# 異常値を範囲内に収める
df_clean['quantity'] = np.clip(df_clean['quantity'], q_low, q_high)
print("数量の異常値を修正しました")

# 単価も同様に
price_low = df_clean['unit_price'].quantile(0.01)
price_high = df_clean['unit_price'].quantile(0.99)
df_clean['unit_price'] = np.clip(df_clean['unit_price'], price_low, price_high)
print("単価の異常値を修正しました")


 外れ値チェック:
数量の正常範囲: 1.0 ～ 10.44999999999959
数量の異常値を修正しました
単価の異常値を修正しました


重複データの削除 - 同じデータを消そう

In [43]:
print(f"\n重複データ: {df_clean.duplicated().sum()}件見つかりました")

# 重複データを削除
before_rows = len(df_clean)
df_clean = df_clean.drop_duplicates()
after_rows = len(df_clean)

print(f"重複データを{before_rows - after_rows}件削除しました")


重複データ: 0件見つかりました
重複データを0件削除しました


データ型の整理 - 数字は数字、文字は文字に

In [44]:
# 数字の列を適切な型に
df_clean['quantity'] = df_clean['quantity'].astype(int)
df_clean['unit_price'] = df_clean['unit_price'].astype(int)
df_clean['total_sales'] = df_clean['total_sales'].astype(int)

print("数値データの型を整理しました")

# カテゴリデータを整理
category_cols = ['product_category', 'customer_gender', 'payment_method', 'region']
for col in category_cols:
    df_clean[col] = df_clean[col].astype('category')

print("カテゴリデータの型を整理しました")

数値データの型を整理しました
カテゴリデータの型を整理しました


前処理前後の比較

In [45]:
def simple_quality_check(df, name):
    """簡単なデータ品質チェック"""
    print(f"\n {name}の品質:")
    print(f"  行数: {len(df):,}行")
    print(f"  欠損値: {df.isnull().sum().sum():,}箇所")
    print(f"  重複: {df.duplicated().sum():,}件")

    # 数値データの基本統計
    if len(df) > 0:
        print(f"  平均単価: {df['unit_price'].mean():.0f}円")
        print(f"  平均数量: {df['quantity'].mean():.1f}個")

# 比較表示
print("=" * 50)
simple_quality_check(df, "クリーニング前")
simple_quality_check(df_clean, "クリーニング後")
print("=" * 50)


 お掃除前の品質:
  行数: 1,000行
  欠損値: 137箇所
  重複: 0件
  平均単価: 30968円
  平均数量: 6.2個

 お掃除後の品質:
  行数: 1,000行
  欠損値: 0箇所
  重複: 0件
  平均単価: 25512円
  平均数量: 5.5個


データをさらに使いやすくする工夫

In [46]:
# 新しい列を作って分析しやすく

# 売上金額でランク分け
df_clean['sales_rank'] = pd.cut(df_clean['total_sales'],
                               bins=[0, 1000, 5000, 10000, 50000, 1000000],
                               labels=['超少額', '少額', '中額', '高額', '超高額'])

# 時間に関する情報を追加
df_clean['order_month'] = df_clean['order_date'].dt.month
df_clean['order_dayofweek'] = df_clean['order_date'].dt.dayofweek
df_clean['is_weekend'] = df_clean['order_dayofweek'].isin([5, 6])

print("分析用の新しい列を作成しました")

✅ 分析用の新しい列を作成しました


文字データをきれいに

In [47]:
# 表記ゆれを統一
df_clean['payment_method'] = df_clean['payment_method'].replace({
    'credit card': 'Credit Card',
    'Credit Card': 'Credit Card',
    'cash': 'Cash',
    'Cash': 'Cash',
    'e-money': 'E-money',
    'E-money': 'E-money',
    'qr code': 'QR Code',
    'QR code': 'QR Code'
})

print("支払方法の表記を統一しました")

✅ 支払方法の表記を統一しました


前処理作業の記録

In [49]:
# どんなお掃除をしたか記録
cleaning_log = """
データクリーニング作業記録

1. 欠損値処理:
   - 数値列: 平均値で補完
   - 文字列: '不明'で補完

2. 外れ値処理:
   - 数量: 上下1%の範囲に収める
   - 単価: 上下1%の範囲に収める

3. 重複処理:
   - 重複行を削除

4. データ型整理:
   - 数値列を適切な型に変換
   - カテゴリ列を整理

5. データ強化:
   - 売上ランク列を追加
   - 時間情報を追加
   - 表記ゆれを統一
"""

print(cleaning_log)


データクリーニング作業記録

1. 欠損値処理:
   - 数値列: 平均値で補完
   - 文字列: '不明'で補完

2. 外れ値処理:
   - 数量: 上下1%の範囲に収める
   - 単価: 上下1%の範囲に収める

3. 重複処理:
   - 重複行を削除

4. データ型整理:
   - 数値列を適切な型に変換
   - カテゴリ列を整理

5. データ強化:
   - 売上ランク列を追加
   - 時間情報を追加
   - 表記ゆれを統一



よくある失敗と対策  
・失敗例1: 元データを壊してしまう  
・失敗例2: 全部削除してしまう  

In [51]:
# ダメな例
# df = df.dropna()  # 元データが壊れる！

# 良い例
# df_clean = df.copy()  # まずコピー
# df_clean = df_clean.dropna()  # コピーを変更

# ダメな例
# df_clean = df.dropna()  # 空欄がある行を全部削除

# 良い例
# 重要な列だけチェックして、必要なら部分削除
# important_cols = ['order_id', 'total_sales']
# df_clean = df.dropna(subset=important_cols)

守るべき3つのルール  
・コピーしてから始める: df_clean = df.copy()

・少しずつ直す: 一気にやらず、1つずつ確認

・結果を確認する: 直したら必ずチェック

段階的に進めよう

In [53]:
# # ステップ1: 欠損値だけ処理
# step1 = df.copy()
# step1 = handle_missing_values(step1)

# # ステップ2: 外れ値処理
# step2 = step1.copy()
# step2 = handle_outliers(step2)

# # ステップ3: 重複処理
# step3 = step2.copy()
# step3 = remove_duplicates(step3)

In [55]:
# 自動化の第一歩
def simple_cleaner(df):
    """初心者向け簡単クリーナー"""
    df_clean = df.copy()

    # 基本的な処理
    df_clean = df_clean.dropna(subset=['total_sales', 'order_id'])  # 重要列の空欄行削除
    df_clean = df_clean.drop_duplicates()  # 重複削除

    # 数値の異常値を直す
    for col in ['quantity', 'unit_price']:
        if col in df_clean.columns:
            low = df_clean[col].quantile(0.01)
            high = df_clean[col].quantile(0.99)
            df_clean[col] = np.clip(df_clean[col], low, high)

    return df_clean

# 使ってみよう
df_clean = simple_cleaner(df)
print("簡単前処理完了！")

 簡単全処理完了！


データフィルタリングと集計

In [ ]:
# 条件によるフィルタリング
filtered_df = df[df['sales'] > 1000]

# グループ化集計（地域別平均売上）
grouped_df = df.groupby('region')['sales'].mean().reset_index()

自動化とファイル操作 (OS, Shutil)

1. ファイルの一括処理

In [ ]:
import os

# ディレクトリ内の全ファイルを処理
for file in os.listdir('path/to/directory'):
    if file.endswith('.csv'):
        file_path = os.path.join('path/to/directory', file)
        # ファイル処理...

API連携 (Requests)

外部サービスからデータを取得する一般的な方法です。

In [ ]:
import requests

# GETリクエストの送信（データ取得）
url = 'https://api.example.com/data'
response = requests.get(url, params={'key': 'your_api_key'})

if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data)

日付処理 (Datetime)

In [3]:
from datetime import datetime, timedelta

# 現在日時の取得
now = datetime.now()

# 日付の計算（30日前）
thirty_days_ago = now - timedelta(days=30)

# 日付のフォーマット
formatted_date = now.strftime('%Y-%m-%d')